# 第二章 OpenAI 函数调用 OpenAI Function Calling

本章主要介绍 OpenAI 的 API 提供的新功能，在最近几个月，OpenAI 对一些新的模型进行了微调，分别是`gpt-3.5-turbo-0613`和`gpt-4-0613`。

假如我们有一个工具函数，在一些特定情况下需要调用，经过微调后，这些模型能够传入新的参数，可以通过这个新的参数来自动判断是否调用工具函数，如果判断需要调用工具函数，会返回这个工具函数和对应的输入参数。


- [一、OpenAI函数新参数](#一openai函数新参数)
  - [1.1 简单例子：得到当前天气](#11-简单例子得到当前天气)  
  - [1.2 新参数：functions](#12-新参数functions) 
  - [1.3 相关提示调用结果](#13-相关提示调用结果)    
  - [1.4 无关提示调用结果](#14-无关提示调用结果)
- [二、Function Call参数模式](#二function-call参数模式)
  - [2.1 自动判断是否调用](#21-自动判断是否调用)    
  - [2.2 强制不调用](#22-强制不调用)    
  - [2.3 强制调用](#23-强制调用)  
- [三、函数调用以及执行函数](#三函数调用以及执行函数)   
- [四、英文版提示](#四英文版提示)

为了大家实验方便，关于以下示例，我们将使用硅基流动的API，于是相关的代码也会修改。

## 一、OpenAI函数新参数

首先我们直接定义使用`OPENAI_API_KEY`，方便后续去调用OpenAI的API接口，并利用其函数。

In [45]:
import os

from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

loaded = load_dotenv(find_dotenv(), override=True)
# 从环境变量中获取 OpenAI API Key 或者直接赋值
API_KEY = os.getenv("API_KEY")

# 如果您使用的是官方 API，就直接用 https://api.siliconflow.cn/v1 就行。
BASE_URL = "https://api.siliconflow.cn/v1"

# 基于openai的OpenAI实例
openai_client = OpenAI(api_key=API_KEY, base_url=BASE_URL, max_retries=3)

### 1.1 简单例子：得到当前天气

首先我们使用 OpenAI 的使用的第一个例子，定义了一个`get_current_weather`的函数，正常来说，获取当前天气是语言模型本身不能完全做到的事情。因此，我们希望能够将语言模型和这样的函数结合起来，以当前的信息来增强它。

在这个函数，我们固定了返回的值，比如温度固定为 22 摄氏度，但在实际应用中，这可能涉及到调用天气   API 或一些外部知识源。

In [46]:
import json

# 定义一个函数，用于获取给定位置的当前天气
def get_current_weather(location, unit="摄氏度"):
    """获取指定位置的当前天气"""
    # 模拟返回相同的天气情况的示例函数
    # 在实际应用环境中，这可以是天气API
    # 创建一个天气信息的字典
    weather_info = {
        "location": location,  # 天气的位置
        "temperature": "22",  # 温度
        "unit": unit,  # 温度单位，默认为摄氏度
        "forecast": ["晴", "多云"],  # 天气预报
    }
    # 将天气信息转换为JSON格式的字符串并返回
    return json.dumps(weather_info, ensure_ascii=False)

### 1.2 新参数：functions

那我们如何将这样的函数传给语言模型呢？OpenAI引入了一个名为`functions`的新参数，通过该参数，我们可以传递一个函数定义列表。由于我们只有一个函数，所以列表中只有一个元素。这是一个JSON对象，具有几个不同的参数。

- name：函数的名称
- description：函数的描述
- parameters：参数对象，里面有一些属性设置
  - type：类型
  - properties：本身是一个对象，传入的是对应的参数的描述。在例子中我们可以看到有两个元素，`location`和`unit`。每个这些元素都有一个类型，即字符串，然后是一个描述。`unit`是一个外部参数的设置，比如这里我们希望它可以是摄氏度或华氏度，所以我们可以在这里定义他的类型和以及枚举参数值。
  - required：必填的参数，比如这里我们需要的参数就是`location`。

在函数定义中，`description`以及在`properties`中的参数非常重要，因为这些将直接传递给语言模型，语言模型将使用这些描述来判断是否使用此函数。

In [47]:
# 定义一个函数
get_current_weather_function = {
    "name": "get_current_weather",
    "description": "获取指定位置的当前天气情况",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "城市和省份，例如：北京，北京市",
            },
            "unit": {"type": "string", "enum": ["摄氏度", "华氏度"]},
        },
        "required": ["location"],
    },
}

### 1.3 相关提示调用结果

接下来我们就可以定义一个有关于天气的问题，如”北京的天气怎么样？“，然后使用 OpenAI 的函数进行调用对话的 API。首先我们得选取较新的模型，如`gpt-3.5-turbo-0613`，然后我们再将上述定义的        function 函数传入，查看最后的响应结果。

从结果上来看，返回的消息的角色是助手，内容为空，而是有一个函数调用参数`function_call`，其中包含两个对象，`name`和`arguments`。`Name`是`get_current_weather`，这与我们传递的函数的名称相同，然后`arguments`是这个 JSON 格式的字典，里面是我们需要的参数。

In [48]:
# 定义输入消息
messages = [
    {
        "role": "user",
        "content": "北京的天气怎么样?"
    }
]

# 调用OpenAI的chat.completions API获取响应
response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function,
        }
    ],
    extra_body = {
            "enable_thinking": False # 不需要思考
        }
)

# 打印响应结果
print(response)

ChatCompletion(id='01975ed43d288630d6e2706cb282814b', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='01975ed43e0be1908c87f2aea31b33d0', function=Function(arguments=' {"location": "北京", "unit": "摄氏度"}', name='get_current_weather'), type='function', index=0)]))], created=1749642657, model='Qwen/Qwen3-8B', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=15, prompt_tokens=198, total_tokens=213, completion_tokens_details=None, prompt_tokens_details=None))


我们也可以看到响应的参数，比如在这里就是两个参数，分别是`location`和`unit`。

In [50]:
# 打印传入参数
response.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(id='01975ed43e0be1908c87f2aea31b33d0', function=Function(arguments=' {"location": "北京", "unit": "摄氏度"}', name='get_current_weather'), type='function', index=0)

如果我们仔细看响应的话，我们会发现内容现在是空的，`function_call`是一个字典，`function_call`中的`arguments`参数本身也是一个 JSON 字典。因此我们可以使用`json.loads`将其加载到 Python 字典中。它返回的参数可以直接传递给我们上述定义的`get_current_weather`函数。

我们会发现，使用 OpenAI 进行函数调用并不直接调用工具函数，我们还是需要去调用工具函数，它只是告诉我们要调用哪个函数，即名称，以及该函数的参数应该是什么。以及由于他并没有去执行函数，所以说，如果我们在使用`json.loads`解码的时候遇到了一些问题，那实际上是模型的问题，所以这一部分可以考虑在写工具函数的时候做一些保护措施，后续也会讨论这一个点。

In [52]:
# 获取response里面的message信息
response_message = response.choices[0].message

response_message

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='01975ed43e0be1908c87f2aea31b33d0', function=Function(arguments=' {"location": "北京", "unit": "摄氏度"}', name='get_current_weather'), type='function', index=0)])

In [54]:
# 打印参数
print(response.choices[0].message.tool_calls[0].function.arguments)

 {"location": "北京", "unit": "摄氏度"}


In [56]:
# 将JSON格式的字符串转换为Python对象
args = json.loads(response_message.tool_calls[0].function.arguments)

# 调用get_current_weather函数并传入参数args
print(get_current_weather(args))

{"location": {"location": "北京", "unit": "摄氏度"}, "temperature": "22", "unit": "摄氏度", "forecast": ["晴", "多云"]}


### 1.4 无关提示调用结果

接下来，我们也探讨一下，如果问的问题与函数无关会产生什么样的，也就是与天气无关，会返回什么样的信息呢？从结果上来看，我们可以返回的内容正常并且没有`function_call`参数，也就是在语言模型中判断不使用工具函数。

In [57]:
# 与天气无关提示调用
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function,
        }
    ],
    extra_body = {
            "enable_thinking": False # 不需要思考
        }
)

print(response)

ChatCompletion(id='01975ed4db6ec2853d9f5235f9f3d2aa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！有什么我可以帮助你的吗？', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1749642697, model='Qwen/Qwen3-8B', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=8, prompt_tokens=195, total_tokens=203, completion_tokens_details=None, prompt_tokens_details=None))


由于编码问题，我们可以单独打印对应的返回的文本信息

In [58]:
print(response.choices[0].message.content)

你好！有什么我可以帮助你的吗？


## 二、Function Call参数模式

`function_call`参数一共有3种模式，我们可以传递其他参数`function_call`以强制模型使用或不使用函数。

1. 默认情况下，它设置为`auto`，也就是模型自行选择。
2. 在第二种模式中，我们可以强制它调用一个函数。如果我们希望始终返回函数
3. 另一种模式是`none`。这会强制语言模型不使用提供的任何函数。

### 2.1 自动判断是否调用

`auto`模式就是大模型自行选择是否要返回参数，这一部分也是默认的，上述所有的方式都是`auto`的模式

### 2.2 强制不调用

`none`模式对于这个例子来说并不重要，因为内容`你好`不需要函数调用，所以我们看到它没有被使用。

In [62]:
# 无关提示强制不调用
messages = [
    {
        "role": "user",
        "content": "你好",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function,
        }
    ],
    tool_choice="none",
    extra_body = {
            "enable_thinking": False # 不需要思考
        }
)

print(response)
print(response.choices[0].message.content)

ChatCompletion(id='01975ed7f17dece281d9d11dbc2178e3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！😊 今天过得怎么样？有什么我可以帮忙的吗？', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1749642899, model='Qwen/Qwen3-8B', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=14, prompt_tokens=13, total_tokens=27, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=None))
你好！😊 今天过得怎么样？有什么我可以帮忙的吗？


那如果我们在需要使用工具的函数时候强制不调用（也就是使用第二种模式）会出现什么结果呢？从结果上来看，他依然有正常的`role`和`content`，但是因为我们强制不调用函数，所以他在试图返回正确的参数，但是并不正确。

In [65]:
# 相关提示强制不调用
messages = [
    {
        "role": "user",
        "content": "北京天气怎么样?",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function,
        }
    ],
    tool_choice="none",
    extra_body = {
            "enable_thinking": False # 不需要思考
        }
)

print(response)
print(response.choices[0].message.content)

ChatCompletion(id='01975ed906937d237899fdd91b6b9641', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='北京的天气会根据季节不同而变化较大，让我为您查询一下当前天气情况。\n\n当前日期是2023年10月10日，根据最新的气象数据，北京今天白天多云转晴，气温较低，平均气温在15°C左右，风力较小，适合户外活动。夜间的天气也会保持晴朗，但气温可能会进一步下降，建议适当增添衣物保暖。\n\n如果您需要更详细的信息，比如具体的天气预报、湿度、风速等，可以告诉我您需要查询的具体时间，我将为您提供更精确的数据。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1749642970, model='Qwen/Qwen3-8B', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=118, prompt_tokens=16, total_tokens=134, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=None))
北京的天气会根据季节不同而变化较大，让我为您查询一下当前天气情况。

当前日期是2023年10月10日，根据最新的气象数据，北京今天白天多云转晴，气温较低，平均气温在15°C左右，风力较小，适合户外活动。夜间的天气也会保持晴朗，但气温可能会进一步下降，建议适当增添衣物保暖。

如果您需要更详细的信息，比如具体的天气预报、湿

### 2.3 强制调用

`function_call`参数的最后一个模式是强制调用函数，方法也是很简单，只要在参数里面传入这个函数的名字即可。在这一部分，我们指定`name`为`get_current_weather`，这将强制它使用`get_current_weather`函数。如果我们查看结果，我们实际上会看到返回了这个`function_call`对象，其中有`name`为`get_current_weather`的一些参数。

为了好玩，这里面传了一个于天气无关的参数，然后强制他使用`get_current_weather`函数，但我们传入的参数中绝对没有任何信息关于它应该如何调用该函数。所以在这里，他编造了北京，北京市的参数，如果我们再次运行它，它会不断地调用北京市的参数。

我们还可以尝试使用不同的模型函数来调用，不同的参数值，不同的输入消息，但是这里面要注意一个事情，首先，函数本身和描述都会计入传递给OpenAI的令牌使用限制。所以如果我们运行这个，我们可以看到返回的提示令牌是95。如果我们注释掉`functions`和`function_call`，我们可以看到提示令牌减少到15。因为OpenAI模型对令牌有限制，因此，在构造要传递给OpenAI的消息时，现在不仅需要注意消息的长度，还需要注意传递的函数的长度。

In [69]:
# 无关提示强制调用函数
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function,
        }
    ],
    tool_choice={
        "type": "function",
        "function": {
            "name": "get_current_weather",
        }
    },
)

print(response)
print(response.choices[0].message.tool_calls[0].function.arguments)

ChatCompletion(id='01975eef34be7cca916e7df1079ed4d0', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='01975eef357747a9fbbd6fbdfc68f496', function=Function(arguments='{"location": "北京", "unit": "摄氏度"}', name='get_current_weather'), type='function', index=0)]))], created=1749644424, model='Qwen/Qwen3-8B', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=14, prompt_tokens=195, total_tokens=209, completion_tokens_details=None, prompt_tokens_details=None))
{"location": "北京", "unit": "摄氏度"}


## 三、函数调用以及执行函数

最后，让我们看一下如何将这些函数调用和实际执行函数调用的结果传递回语言模型。这很重要，因为通常我们希望使用语言模型确定要调用的函数，然后运行该函数，但然后将其传递回语言模型以获得最终响应。

我们会经过这样的流程，首先我们问问题得到一个带有`function call`参数的响应，然后我们将这个消息添加到我们的消息列表中。然后我们可以模拟使用语言模型提供的参数调用`get_current_weather`函数，并且保存到一个变量`observation`中。接着我们定义一个新的消息列表，表示刚刚调用函数的结果，这里面有一个重要的点就是`role`等于`function`，也就是告诉语言模型这是调用函数的响应。除此之外，还传递函数的名称`name`以及`content`变量，设为上述计算的`observation`。

如果我们然后使用这个消息列表调用语言模型，我们可以看到语言模型回答的非常好：北京的天气目前是22摄氏度，天气以晴天为主，也有多云的情况。

In [70]:
# 与天气无关提示调用
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function,
        }
    ],
    tool_choice="auto",
)

print(response)

ChatCompletion(id='01975eefd60f2bb5f11319f00236f423', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！有什么我可以帮助你的吗？', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1749644465, model='Qwen/Qwen3-8B', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=8, prompt_tokens=195, total_tokens=203, completion_tokens_details=None, prompt_tokens_details=None))


由于编码问题，我们可以单独答应对应的返回的文本信息

In [71]:
print(response.choices[0].message.content)

你好！有什么我可以帮助你的吗？


## 四、英文版提示

**1.1 简单例子：得到当前天气**

In [73]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather_en(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

**1.2 定义function函数**

In [75]:
# define a function
get_current_weather_function_en = {
    "name": "get_current_weather_en",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
    },
}

**1.3 相关提示调用结果**

In [92]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function_en,
        }
    ]
)

print(response.to_json())

{
  "id": "01975ef80ee965c31cf9a0565b61841d",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "content": "",
        "role": "assistant",
        "tool_calls": [
          {
            "id": "01975ef80fd2ee2b9ab45f08a6466f54",
            "function": {
              "arguments": " {\"location\": \"Boston\", \"unit\": \"celsius\"}",
              "name": "get_current_weather_en"
            },
            "type": "function",
            "index": 0
          }
        ]
      }
    }
  ],
  "created": 1749645004,
  "model": "Qwen/Qwen3-8B",
  "object": "chat.completion",
  "system_fingerprint": "",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 204,
    "total_tokens": 218
  }
}


**1.4 无关提示调用结果**

In [85]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function_en,
        }
    ]
)

print(response.to_json())

{
  "id": "01975ef5df8224722bd22f40aaa96f33",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello! How can I assist you today? 😊",
        "role": "assistant"
      }
    }
  ],
  "created": 1749644861,
  "model": "Qwen/Qwen3-8B",
  "object": "chat.completion",
  "system_fingerprint": "",
  "usage": {
    "completion_tokens": 11,
    "prompt_tokens": 198,
    "total_tokens": 209
  }
}


**2.1 自动判断是否调用**

自动判断是否调用函数，判断提示是否和函数相关

In [93]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function_en,
        }
    ],
    tool_choice="auto",
)

print(response.to_json())

{
  "id": "01975ef83e9fa0c99cdbfaeb204161d2",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello! How can I assist you today? 😊",
        "role": "assistant"
      }
    }
  ],
  "created": 1749645016,
  "model": "Qwen/Qwen3-8B",
  "object": "chat.completion",
  "system_fingerprint": "",
  "usage": {
    "completion_tokens": 11,
    "prompt_tokens": 198,
    "total_tokens": 209
  }
}


**2.2 强制不调用**

无关提示强制不调用，结果正常

In [96]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function_en,
        }
    ],
    tool_choice="none",
)

print(response.to_json())

{
  "id": "01975ef9451c1aa86d425c3d80a5d1ac",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "content": "",
        "role": "assistant",
        "tool_calls": [
          {
            "id": "01975ef945ca50049e1dbe7da752ba7c",
            "function": {
              "arguments": "{\"location\": \"San Francisco, CA\", \"unit\": \"celsius\"}",
              "name": "get_current_weather_en"
            },
            "type": "function",
            "index": 0
          }
        ]
      }
    }
  ],
  "created": 1749645084,
  "model": "Qwen/Qwen3-8B",
  "object": "chat.completion",
  "system_fingerprint": "",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 198,
    "total_tokens": 214
  }
}


**2.3 强制调用**

强制无关问题的调用

In [97]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function_en,
        }
    ],
    tool_choice={
        "type": "function",
        "function": {
            "name": "get_current_weather_en",
        }
    },
)

print(response.to_json())

{
  "id": "01975ef9fae6f31255f61e2d3a372a5d",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "content": "",
        "role": "assistant",
        "tool_calls": [
          {
            "id": "01975ef9fb8b081a97c57453020fcba4",
            "function": {
              "arguments": "{\"location\": \"San Francisco, CA\", \"unit\": \"fahrenheit\"}",
              "name": "get_current_weather_en"
            },
            "type": "function",
            "index": 0
          }
        ]
      }
    }
  ],
  "created": 1749645130,
  "model": "Qwen/Qwen3-8B",
  "object": "chat.completion",
  "system_fingerprint": "",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 198,
    "total_tokens": 214
  }
}


强制有关问题调用，结果和`auto`一样

In [100]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]

response = openai_client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=messages,
    extra_body={
        "enable_thinking": False  # 不需要思考
    },
    tools=[
        {
            "type": "function",
            "function": get_current_weather_function_en,
        }
    ],
    tool_choice={
        "type": "function",
        "function": {
            "name": "get_current_weather_en",
        }
    },
)
print(response.to_json())

{
  "id": "01975efafd19bb0914036e8af020146b",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "content": "",
        "role": "assistant",
        "tool_calls": [
          {
            "id": "01975efafd553f6ceb14bcf283bdb68d",
            "function": {
              "arguments": "{\n  \"location\": \"Boston\",\n  \"unit\": \"fahrenheit\"\n}",
              "name": "get_current_weather_en"
            },
            "type": "function",
            "index": 0
          }
        ]
      }
    }
  ],
  "created": 1749645196,
  "model": "Qwen/Qwen3-8B",
  "object": "chat.completion",
  "system_fingerprint": "",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 204,
    "total_tokens": 221
  }
}
